In [14]:
!wget https://raw.githubusercontent.com/dayangalyssa/InfoKilat/main/notebook/Ekstraksi_dan_Seleksi_Fitur.ipynb

--2025-05-31 10:18:47--  https://raw.githubusercontent.com/dayangalyssa/InfoKilat/main/notebook/Ekstraksi_dan_Seleksi_Fitur.ipynb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 101530 (99K) [text/plain]
Saving to: ‘Ekstraksi_dan_Seleksi_Fitur.ipynb.1’

Ekstraksi_dan_Selek 100%[===================>]  99.15K  --.-KB/s    in 0.09s   

2025-05-31 10:18:47 (1.04 MB/s) - ‘Ekstraksi_dan_Seleksi_Fitur.ipynb.1’ saved [101530/101530]



In [28]:
import nbformat

with open("Ekstraksi_dan_Seleksi_Fitur.ipynb") as f:
    nb = nbformat.read(f, as_version=4)
    print(nb['cells'][0]['source'])

import pandas as pd

url = 'https://raw.githubusercontent.com/dayangalyssa/InfoKilat/main/data/preprocessed_sentences.csv'
df = pd.read_csv(url)

df.head(10)


#Implementasi Model

In [16]:
!pip install transformers sentencepiece --quiet

In [17]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load tokenizer dan model
tokenizer = AutoTokenizer.from_pretrained("cahya/bert2bert-indonesian-summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("cahya/bert2bert-indonesian-summarization")

# Fungsi untuk merangkum teks
def summarize(text):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Contoh penggunaan
text = "Masukkan teks panjang dalam bahasa Indonesia di sini untuk dirangkum."
print(summarize(text))

masukkan teks panjang dalam bahasa indonesia di bahasa indonesia, rendarizetta merilis buku terbaru dalam bahasa inggris. dalam buku ini, summarize mengangkat isu tanah air di tanah air yang sedang berbenah.


In [18]:
def ringkas_teks_indo(text, max_input_length=512, max_output_length=100):
    # Tambah prefix khusus untuk tugas summarization
    input_text = "ringkasan: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=max_input_length, truncation=True)

    summary_ids = model.generate(inputs, max_length=max_output_length, min_length=20, length_penalty=2.0, num_beams=4, early_stopping=True)

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [19]:
import re

# Bersihkan teks
def bersihkan(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'<.*?>', '', text)
    return text.strip()

# Simpan hasil ringkasan
indot5_summary_dict = {}

for doc_id in df['document_id'].unique():
    doc_text = ' '.join(df[df['document_id'] == doc_id]['original_sentence'].dropna().astype(str).tolist())
    doc_text = bersihkan(doc_text)

    if len(doc_text) < 50:
        continue

    try:
        summary = ringkas_teks_indo(doc_text[:1024])  # batas aman
        indot5_summary_dict[doc_id] = summary
    except Exception as e:
        indot5_summary_dict[doc_id] = "(Ringkasan gagal)"

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [20]:
indot5_summary_df = pd.DataFrame.from_dict(indot5_summary_dict, orient='index', columns=['indot5_summary'])
indot5_summary_df.reset_index(inplace=True)
indot5_summary_df.rename(columns={'index': 'document_id'}, inplace=True)

display(indot5_summary_df.head(100))


,document_id,indot5_summary
0,0,anggota komisi vii dpr ri rofik hananto menyay...
1,1,"kebakaran depo pertamina di plumpang, jakarta ..."
2,2,delegasi paguyuban first travel indonesia mene...
3,3,tim kedokteran dan kesehatan ( dokkes ) polri ...
4,4,ketua mpr ri sekaligus ketua umum ikatan motor...
5,5,wakil presiden ma ruf meninjau lokasi kebakara...
6,6,menteri keuangan ri sri mulyani indrawati berk...
7,7,keputusan pengadilan negeri jakarta pusat meng...
8,8,markas besar kepolisian republik indonesia tur...
9,9,"kawasan penyangga depo pertamina plumpang, jak..."


In [21]:
indot5_summary_df.to_csv('indot5_summary.csv', index=False)
full_df = pd.read_csv('indot5_summary.csv')
print(full_df)


    document_id                                     indot5_summary
0             0  anggota komisi vii dpr ri rofik hananto menyay...
1             1  kebakaran depo pertamina di plumpang, jakarta ...
2             2  delegasi paguyuban first travel indonesia mene...
3             3  tim kedokteran dan kesehatan ( dokkes ) polri ...
4             4  ketua mpr ri sekaligus ketua umum ikatan motor...
5             5  wakil presiden ma ruf meninjau lokasi kebakara...
6             6  menteri keuangan ri sri mulyani indrawati berk...
7             7  keputusan pengadilan negeri jakarta pusat meng...
8             8  markas besar kepolisian republik indonesia tur...
9             9  kawasan penyangga depo pertamina plumpang, jak...
10           10  presiden joko widodo memerintahkan menteri bum...
11           11  komisi yudisial diminta menggali motif hakim p...
12           12  ketua umum partai nasional demokrat surya palo...
13           13  ketua mpr ri yandri susanto optimis kpu dapat

## Evaluasi

In [22]:
!pip install -q scikit-learn pandas

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
manual_url = "https://docs.google.com/spreadsheets/d/1zcNv1AcX6HwV6Bsj_si15dez7PAKJWi6Up9z9Yy9XeM/export?format=csv"
manual_df = pd.read_csv(manual_url)
manual_df.columns

Index(['document_id', 'manual_summary'], dtype='object')

In [24]:
# Pastikan document_id bertipe string
manual_df['document_id'] = manual_df['document_id'].astype(str)
full_df['document_id'] = full_df['document_id'].astype(str)

# Merge berdasarkan document_id
merged_df = pd.merge(full_df, manual_df[['document_id', 'manual_summary']], on='document_id', how='inner')

In [25]:
def get_similarity(row):
    texts = [str(row['indot5_summary']), str(row['manual_summary'])]
    tfidf = TfidfVectorizer().fit_transform(texts)
    return cosine_similarity(tfidf[0:1], tfidf[1:2])[0][0]

merged_df['cosine_similarity'] = merged_df.apply(get_similarity, axis=1)

In [26]:
merged_df[['document_id', 'indot5_summary', 'manual_summary', 'cosine_similarity']].head(50)

,document_id,indot5_summary,manual_summary,cosine_similarity
0,0,anggota komisi vii dpr ri rofik hananto menyay...,Anggota Komisi VII DPR RI Rofik Hananto menyay...,0.712392
1,1,"kebakaran depo pertamina di plumpang, jakarta ...",Presiden Joko Widodo atau Jokowi memerintahkan...,0.233199
2,2,delegasi paguyuban first travel indonesia mene...,Wakil Ketua MPR RI Dr. H. M. Hidayat Nur Wahid...,0.207144
3,3,tim kedokteran dan kesehatan ( dokkes ) polri ...,Tim Kedokteran dan Kesehatan (Dokkes) Polri te...,1.000000
4,4,ketua mpr ri sekaligus ketua umum ikatan motor...,Ketua MPR RI sekaligus Ketua Umum Ikatan Motor...,1.000000
5,5,wakil presiden ma ruf meninjau lokasi kebakara...,"Pada 3 Maret 2023, kebakaran hebat melanda Dep...",0.405408
6,6,menteri keuangan ri sri mulyani indrawati berk...,Memperingati 14 tahun PT Sarana Multi Infrastr...,0.310782
7,7,keputusan pengadilan negeri jakarta pusat meng...,Pengadilan Negeri atau PN Jakarta Pusat memutu...,0.154640
8,8,markas besar kepolisian republik indonesia tur...,Markas Besar Kepolisian Republik Indonesia tur...,0.593011
9,9,"kawasan penyangga depo pertamina plumpang, jak...",Buffer zone atau kawasan penyangga yang memisa...,0.516214


In [27]:
average_cosine = merged_df['cosine_similarity'].mean()
average_cosine = average_cosine*100
print(f"Average Cosine Similarity: {average_cosine:.0f}%")


Average Cosine Similarity: 54%
